# Recommendation Systems for Hearthstone Decks

In [ ]:
!pip install pyreclab

In [ ]:
import pandas as pd
import pyreclab
import seaborn as sns
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('archive/data.csv')
data.head()

## Random

## Most Popular

## Otro